In [1]:
#aysenur

In [2]:
import pytorch_lightning as pl

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger
import wandb

In [3]:
#from DataPreparation import get_data
from TimeSeriesLearningUtils import TimeSeriesDataset, CosineWarmupScheduler, get_data
from LSTMModel import LSTM_based_classification_model

In [4]:
def name_model(config):
    name =[]
    if len(config["currency_list"])  > 1:
        name.append("multi_task_" + "_".join(config["currency_list"]))
    else:
        name.append(config["currency_list"][0])
        
    if config["indicators"] or config["imfs"] or config ["ohlv"]:
        name.append("multi_variate")
    
    lstm = "stack_lstm" if len(config["lstm_hidden_sizes"]) > 1 else "lstm"
    name.append(lstm)
    
    name.append(config["pred_frequency"])
    classification = "multi_clf" if config["n_classes"] > 2 else "binary_clf"
    name.append(classification)
    
    return "_".join(name)

In [22]:
WANDBPROJECT = "deneme"

config = {"window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "data_frequency": "6h", 
          "pred_frequency": "6h",
          "ma_period": 10,
          "neutral_quantile": 0.33,
          "batch_size": 16,
          "bidirectional": True, 
          "n_classes": 2,
          "currency_list": ['BTC', 'ETH', 'LTC'],
          "remove_trend": True,
          "lstm_hidden_sizes": [128, 128, 128],
          "loss_weight_calculate": True, 
          "indicators": True, 
          "imfs": False,
          "ohlv": True}

In [18]:
name_model(config)

'multi_task_BTC_ETH_LTC_multi_variate_stack_lstm_6h_binary_clf'

In [19]:
###########

In [23]:
MODEL_NAME = name_model(config)

CURRENCY_LST = config["currency_list"]
N_CLASSES = config["n_classes"]
LSTM_HIDDEN_SIZES = config["lstm_hidden_sizes"]
BIDIRECTIONAL = config["bidirectional"]
REMOVE_TREND =config["remove_trend"]
LOSS_WEIGHT_CALCULATE = config["loss_weight_calculate"]

TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = config["dataset_percentages"] 
WINDOW_SIZE = config["window_size"]
DATA_FREQUENCY = config["data_frequency"]
PRED_FREQUENCY = config["pred_frequency"]
MA_PERIOD = config["ma_period"]
NEUTRAL_QUANTILE = config["neutral_quantile"] if N_CLASSES > 2 else 0 
BATCH_SIZE= config["batch_size"]
INDICATORS = config["indicators"]
IMFS = config["imfs"]
OHLV = config["ohlv"]
#####

X, y, features, dfs = get_data(currency_lst = CURRENCY_LST,
                               data_frequency =DATA_FREQUENCY,
                               pred_frequency = PRED_FREQUENCY,
                               n_classes = N_CLASSES,
                               window_size=WINDOW_SIZE,
                               neutral_quantile = NEUTRAL_QUANTILE,
                               log_price=True,
                               remove_trend=REMOVE_TREND,
                               ma_period=MA_PERIOD,
                               include_indicators = INDICATORS,
                               include_imfs = IMFS, 
                               open_high_low_volume = OHLV, 
                               drop_missing=True)


INPUT_FEATURE_SIZE = X.shape[-1]

train_dataset, val_dataset, test_dataset = [TimeSeriesDataset(CURRENCY_LST, 
                                                          X, 
                                                          y, 
                                                          dtype, 
                                                          TRAIN_PERCENTAGE, 
                                                          VAL_PERCENTAGE, 
                                                          TEST_PERCENTAGE, 
                                                          WINDOW_SIZE) for dtype in ['train', 'val', 'test']]

config["dataset_sizes"] = [len(train_dataset), len(val_dataset), len(test_dataset)]

In [24]:
X.shape

(3, 5340, 88)

In [25]:
wandb.init(project="deneme",
           config=config,
           entity='multi_task_price_prediction',
           name = MODEL_NAME)

logger = WandbLogger()
#     #logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstm_multi_task")

model = LSTM_based_classification_model(
    train_dataset = train_dataset,
     val_dataset = val_dataset,
     test_dataset = test_dataset,
     calculate_loss_weights = LOSS_WEIGHT_CALCULATE,
     currencies = CURRENCY_LST,
     num_classes = N_CLASSES,
     window_size = WINDOW_SIZE,
     input_size = INPUT_FEATURE_SIZE,
     batch_size=BATCH_SIZE,
     lstm_hidden_sizes = LSTM_HIDDEN_SIZES,
     bidirectional = BIDIRECTIONAL)

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.003,
   patience=20,
   verbose=True,
   mode='min'
)
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='../output/',
    filename=MODEL_NAME +'-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)

trainer = pl.Trainer(gpus=-1, 
                     max_epochs= 80,
                     logger = logger, 
                     callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model)

trainer.test(ckpt_path = checkpoint_callback.best_model_path)
wandb.finish()

wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


TypeError: __init__() missing 1 required positional argument: 'last_layer_fsz'

In [1]:
#denemeleri tamamlandı experiment_lstm.py dosyasına yazıldı işlemler
def experiment(script):
    !python ../pipelines/multi_task_price_change_prediction/experiment_lstm.py $script

In [2]:
CONFIG = {#fix for this project
          "window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "frenquency": "D", 
          "neutral_quantile": 0.33,
          "batch_size": 16, 
    "imfs": False, 
    "ohlv": False, 
    "remove_trend": True}

In [3]:
from sklearn.model_selection import ParameterGrid
param_grid = {"n_classes": [2,3],
          "currency_list": [['BTC'], ['ETH'], ['LTC'], ['BTC', 'ETH'],  ['BTC', 'ETH', 'LTC']],
          
          "lstm_hidden_sizes": [[128, 128, 128]],
          "loss_weight_calculate": [True, False],
          "indicators": [True, False],
             "bidirectional":[True, False]}

In [4]:
for c in (ParameterGrid(param_grid)):
    config = CONFIG.copy()
    config.update(c)
    script = "--currency-list " + " ".join([i for i in config["currency_list"]])
    script += " --lstm-list " + " ".join([str(i) for i in config["lstm_hidden_sizes"]])
    script += " -trend " + str(1 if config["remove_trend"] else 0)
    script += " -indicators " + str(1 if config["indicators"] else 0)
    script += " -ohlv " + str(1 if config["ohlv"] else 0)
    script += " -imfs " + str(1 if config["imfs"] else 0)
    script += " -bidirectional " + str(1 if config["bidirectional"] else 0)
    script += " -classes " + str(config["n_classes"] )
    script += " -weight " + str(1 if config["loss_weight_calculate"] else 0) 

    experiment(script)

/home/aysenurk/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/aysenurk/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-07-10 09:37:19.420059: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: Tracking run with wandb version 0.10.33
wandb: Syncing run single_task_BTC_stack_lstm_binary_classification
wandb:  View project at https://wandb.ai/aysenurk/price_change_v3
wandb:  View run at https://wandb.ai/aysenurk/price_change_v3/runs/2fwbi3d3
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_093718-2fwbi3d3
wandb: Run `

Epoch 17: 100%|█| 21/21 [00:00<00:00, 32.95it/s, loss=0.424, v_num=i3d3, BTC_val
Epoch 18:  95%|▉| 20/21 [00:00<00:00, 28.53it/s, loss=0.428, v_num=i3d3, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 18: 100%|█| 21/21 [00:00<00:00, 28.81it/s, loss=0.428, v_num=i3d3, BTC_val
Epoch 19:  95%|▉| 20/21 [00:00<00:00, 33.93it/s, loss=0.427, v_num=i3d3, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 19: 100%|█| 21/21 [00:00<00:00, 34.10it/s, loss=0.427, v_num=i3d3, BTC_val
Epoch 20:  95%|▉| 20/21 [00:00<00:00, 31.47it/s, loss=0.415, v_num=i3d3, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 20: 100%|█| 21/21 [00:00<00:00, 31.15it/s, loss=0.415, v_num=i3d3, BTC_val
Epoch 21:  95%|▉| 20/21 [00:00<00:00, 32.88it/s, loss=0.421, v_num=i3d3, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 21: 100%|█| 21/21 [00:00<00:00, 32.68it/s, loss=0.421, v_num=i3d3, BTC_val
Epoch 22:  95%|▉| 20/21 [00:00<00:00, 31.34it/s, loss=0.416, v_num=i3d3, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 22: 100%|█| 21/21 [00:00<00:0

/home/aysenurk/anaconda3/lib/python3.8/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `F1` was deprecated since v1.3.0 in favor of `torchmetrics.classification.f_beta.F1`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
/home/aysenurk/anaconda3/lib/python3.8/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.classification.accuracy.Accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type        | Params
----------------------------------------------------
0  | lstm_1             | LSTM        | 219 K 
1  | batch_norm1        | BatchNorm2d | 512   
2  | lstm_2             | LSTM        | 395 K 
3  | batch_norm2        | BatchNorm2d | 512   
4  | lstm_3             | LSTM        | 395 K 
5  

Epoch 22: 100%|█| 21/21 [00:00<00:00, 32.58it/s, loss=0.668, v_num=0sjq, BTC_val
Epoch 23:  95%|▉| 20/21 [00:00<00:00, 32.48it/s, loss=0.653, v_num=0sjq, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 23: 100%|█| 21/21 [00:00<00:00, 32.74it/s, loss=0.653, v_num=0sjq, BTC_val
Epoch 24:  95%|▉| 20/21 [00:00<00:00, 34.16it/s, loss=0.63, v_num=0sjq, BTC_val_
Validating: 0it [00:00, ?it/s]
Epoch 24: 100%|█| 21/21 [00:00<00:00, 34.52it/s, loss=0.63, v_num=0sjq, BTC_val_
Epoch 25:  95%|▉| 20/21 [00:00<00:00, 32.84it/s, loss=0.644, v_num=0sjq, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 25: 100%|█| 21/21 [00:00<00:00, 33.09it/s, loss=0.644, v_num=0sjq, BTC_val
Epoch 26:  95%|▉| 20/21 [00:00<00:00, 31.47it/s, loss=0.638, v_num=0sjq, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 26: 100%|█| 21/21 [00:00<00:00, 31.79it/s, loss=0.638, v_num=0sjq, BTC_val
Epoch 27:  95%|▉| 20/21 [00:00<00:00, 33.67it/s, loss=0.623, v_num=0sjq, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 27: 100%|█| 21/21 [00:00<00:0


   | Name               | Type        | Params
----------------------------------------------------
0  | lstm_1             | LSTM        | 219 K 
1  | batch_norm1        | BatchNorm2d | 512   
2  | lstm_2             | LSTM        | 395 K 
3  | batch_norm2        | BatchNorm2d | 512   
4  | lstm_3             | LSTM        | 395 K 
5  | batch_norm3        | BatchNorm2d | 512   
6  | dropout            | Dropout     | 0     
7  | linear1            | ModuleList  | 32.9 K
8  | activation         | ReLU        | 0     
9  | output_layers      | ModuleList  | 258   
10 | cross_entropy_loss | ModuleList  | 0     
11 | f1_score           | F1          | 0     
12 | accuracy_score     | Accuracy    | 0     
----------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.177     Total estimated model params size (MB)
Validation sanity check: 0it [00:00, ?it/s]/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_l

...

Epoch 0: 100%|█| 19/19 [00:00<00:00, 20.47it/s, loss=0.708, v_num=sepm, BTC_val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved. New best score: 0.733
Epoch 0: 100%|█| 19/19 [00:00<00:00, 19.96it/s, loss=0.708, v_num=sepm, BTC_val_
                                                                                /home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  warning_cache.deprecation(
Epoch 1:  95%|▉| 18/19 [00:00<00:00, 20.32it/s, loss=0.674, v_num=sepm, BTC_val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type        | Params
----------------------------------------------------
0  | lstm_1             | LSTM        | 67.1 K
1  | batch_norm1        | BatchNorm2d | 256   
2  | lstm_2             | LSTM        | 132 K 
3  | batch_norm2        | BatchNorm2d | 256   
4  | lstm_3             | LSTM        | 132 K 
5  | batch_norm3        | BatchNorm2d | 256   
6  | dropout            | Dropout     | 0     
7  | linear1            | ModuleList  | 8.3 K 
8  | activation         | ReLU        | 0     
9  | output_layers      | ModuleList  | 195   
10 | cross_entropy_loss | ModuleList  | 0     
11 | f1_score           | F1          | 0     
12 | accuracy_score     | Accuracy    | 0     
----------------------------------------------------
340 K     Trainable params
0         Non-trainable params
340 K     Total params
1.362     Total estimated model params 

wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-07-10 10:59:20.723800: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: Tracking run with wandb version 0.10.33
wandb: Syncing run multi_task_BTC_ETH_LTC_stack_lstm_binary_classification
wandb:  View project at https://wandb.ai/aysenurk/price_change_v3
wandb:  View run at https://wandb.ai/aysenurk/price_change_v3/runs/26vxq76q
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_105855-26vxq76q
wandb: Run `wandb offline` to turn off syncing.

/home/aysenurk/anaconda3/lib/python3.8/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `F1` was deprecated since v1.3.0 in favor of `torchmetrics.classification.f_beta.F1`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
/home/aysenurk/anaconda3/lib/python3.8/site-

wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-07-10 11:00:15.586660: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: Tracking run with wandb version 0.10.33
wandb: Syncing run multi_task_BTC_ETH_LTC_stack_lstm_multi_classification
wandb:  View project at https://wandb.ai/aysenurk/price_change_v3
wandb:  View run at https://wandb.ai/aysenurk/price_change_v3/runs/2ma0mobt
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_110014-2ma0mobt
wandb: Run `wandb offline` to turn off syncing.

/home/aysenurk/anaconda3/lib/python3.8/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `F1` was deprecated since v1.3.0 in favor of `torchmetrics.classification.f_beta.F1`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
/home/aysenurk/anaconda3/lib/python3.8/site-p

Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Monitored metric val_loss did not improve in the last 20 records. Best score: 0.713. Signaling Trainer to stop.
Epoch 55: 100%|█| 19/19 [00:01<00:00, 18.83it/s, loss=0.721, v_num=mobt, BTC_val
Epoch 55: 100%|█| 19/19 [00:01<00:00, 18.76it/s, loss=0.721, v_num=mobt, BTC_val
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Testing: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 27.33it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.66666668653488